In [0]:
# ==========================================================
# UDV - HM_CAMPEONES
# Proyecto: Liga 1 Perú
# Autor: Oscar García Del Águila
# ==========================================================

from env_setup import *
from pyspark.sql import SparkSession

from utils_liga1 import (
    setup_adls,
    get_dbutils,
    generar_udv_json,
    is_dataframe_empty,
    get_predecesor,
    get_pipeline_params,
    get_yaml_from_param,
    write_delta_udv,
    log,
    read_udv_table,
    get_pipeline,
)

from hm_campeones import carga_final_hm_campeones
import traceback

# ----------------------------------------------------------
# CONFIGURACIÓN INICIAL
# ----------------------------------------------------------

log("Inicio de ejecución del pipeline UDV", "INFO", "hm_campeones")

spark = SparkSession.builder.getOrCreate()
setup_adls()
dbutils = get_dbutils()

In [0]:

# ----------------------------------------------------------
# PARÁMETROS Y PREDECESORES
# ----------------------------------------------------------
try:
    # Para pruebas en notebook (ADF sobreescribe este valor)
    dbutils.widgets.text("prm_pipelineid", "22")  # ajusta el 17 al id real de hm_campeones si fuera otro
    prm_pipelineid = int(dbutils.widgets.get("prm_pipelineid"))

    # Info del pipeline (nombre lógico de la entidad UDV)
    pipeline_info = get_pipeline(prm_pipelineid)
    entity_name   = pipeline_info["pipeline"]  # 'hm_campeones'

    # Predecesores (tb_param_predecesor)
    dict_predecesores = get_predecesor(prm_pipelineid)

    # Esperados para CAMPEONES:
    #  - md_catalogo_equipos : tabla UDV ya construida
    #  - campeones           : entidad RAW (JSON FotMob)
    prm_tabla_md_catalogo_equipos = dict_predecesores["md_catalogo_equipos"]["RutaTabla"]
    entity_raw_campeones          = dict_predecesores["campeones"]["RutaTabla"]

    # Parámetros UDV (tb_param_pipeline_param)
    dict_params     = get_pipeline_params(prm_pipelineid)
    prm_capa_udv    = dict_params["CAPA_UDV"]
    prm_ruta_base   = dict_params["RUTA_BASE"]
    prm_ruta_tabla  = dict_params["RUTA_TABLA"]
    prm_formato     = dict_params["FORMATO_SALIDA"]
    prm_schema_tb   = dict_params["SCHEMA_TABLA"]
    prm_tabla_output = dict_params["NOMBRE_TABLA"]
    prm_ruta_yaml   = dict_params["YAML_PATH"]

    log("Parámetros cargados correctamente", "INFO", entity_name)

except Exception as e:
    log(f"Error al cargar parámetros o predecesores: {e}", "ERROR", "hm_campeones")
    print(traceback.format_exc())
    raise

In [0]:
# ----------------------------------------------------------
# LECTURA YAML
# ----------------------------------------------------------
try:
    yaml_conf = get_yaml_from_param(prm_ruta_yaml)

    # Sección hm_campeones dentro del YAML
    prm_campo_json        = yaml_conf[entity_name]["campo_json"]
    prm_cols_raw_campeones_hm = yaml_conf[entity_name]["cols_raw_campeones_hm"]
    prm_schema_hm         = yaml_conf[entity_name]["schema_hm"]

    # Nota: si en el YAML tienes también fecha_ddMMyyyy, numeric_no_parse, etc.,
    # se pueden ignorar aquí sin problema si la función hm_campeones.py no los usa.
    log("YAML cargado correctamente", "INFO", entity_name)

except Exception as e:
    log(f"Error al leer YAML {prm_ruta_yaml}: {e}", "ERROR", entity_name)
    print(traceback.format_exc())
    raise

In [0]:

# ----------------------------------------------------------
# LECTURA PREDECESORES (MD_CATALOGO_EQUIPOS + RAW CAMPEONES)
# ----------------------------------------------------------
try:
    log("Lectura desde UDV/RAW", "INFO", entity_name)

    # 1) MD_CATALOGO_EQUIPOS (ej: 'tb_udv.md_catalogo_equipos')
    df_md_catalogo_equipos = read_udv_table(prm_tabla_md_catalogo_equipos)
    if is_dataframe_empty(df_md_catalogo_equipos):
        raise Exception(f"No se encontró data en la tabla: {prm_tabla_md_catalogo_equipos}")

    # 2) RAW CAMPEONES (JSON FotMob) desde tbl_paths (flg_udv = 'N')
    df_raw_campeones = generar_udv_json(
        entidad=entity_raw_campeones,  # 'campeones'
        campo_json=prm_campo_json      # normalmente 'data'
    )

    if is_dataframe_empty(df_raw_campeones):
        raise Exception(f"No se encontró data en RAW para la entidad: {entity_raw_campeones}")

    log("Predecesores completados correctamente", "INFO", entity_name)

except Exception as e:
    log(f"Error en lectura de predecesores/RAW: {e}", "ERROR", entity_name)
    print(traceback.format_exc())
    raise

log("Finalización de la etapa de lectura", "INFO", entity_name)

In [0]:

# ----------------------------------------------------------
# EJECUCIÓN PRINCIPAL
# ----------------------------------------------------------
try:
    log("Inicio de Ejecución Principal", "INFO", entity_name)

    # Aquí toda la lógica de negocio está encapsulada en hm_campeones.py
    # y NO usamos withColumn, distinct, collect, lambda, etc.
    df_final_hm = carga_final_hm_campeones(
        df_md_catalogo_equipos=df_md_catalogo_equipos,
        df_raw_campeones=df_raw_campeones,
        prm_cols_raw_campeones_hm=prm_cols_raw_campeones_hm,
        prm_schema_hm=prm_schema_hm,
    )

    if is_dataframe_empty(df_final_hm):
        log("df_final_hm vacío, no se realizará escritura en UDV", "WARN", entity_name)
    else:
        log(
            "Escribiendo HM_CAMPEONES con overwrite dinámico por partición [periodo]",
            "INFO",
            entity_name,
        )

        write_delta_udv(
            spark,
            df_final_hm,
            prm_schema_tb,            # ej: 'tb_udv'
            prm_tabla_output,         # ej: 'hm_campeones'
            mode="overwrite",
            partition_by=["periodo"],
            overwrite_dynamic_partition=True,
        )

        # display(df_final_hm)  # opcional para pruebas en notebook
        log(
            "Escritura HM_CAMPEONES completada con overwrite dinámico",
            "SUCCESS",
            entity_name,
        )

except Exception as e:
    log(f"Error en ejecución principal: {e}", "ERROR", entity_name)
    print(traceback.format_exc())
    raise

log("Finalización del pipeline UDV", "INFO", entity_name)